In [1]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline 
import geopy
from geopy import Nominatim
from geopy.extra.rate_limiter import RateLimiter

housing = pd.read_csv('Ames_HousePrice.csv', index_col=0)
address = pd.read_csv('Ames Real Estate Data.csv')

/Users/Matt/opt/anaconda3/lib/python3.7/site-packages/statsmodels/tools/_testing.py:19: FutureWarning: pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.
  import pandas.util.testing as tm
/Users/Matt/opt/anaconda3/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3063: DtypeWarning: Columns (36,40,81,87) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [ ]:
#Remove the duplicated row (index 1 and index 817 are identical)
housing = housing.drop_duplicates()
housing.shape

In [ ]:
#Check that all PIDs are in the reference set
print(housing.PID[housing.PID.isin(address.GeoRefNo)].shape)
#20 are missing
housing.PID[~housing.PID.isin(address.GeoRefNo)]


In [ ]:
#Subset housing houses that have a match
housing = housing[housing.PID.isin(address.GeoRefNo)]

#Subset address on properties that have a match in housing, dropping extraneous columsn and duplicates
subset = address[address.GeoRefNo.isin(housing.PID)]
subset = subset[['GeoRefNo','Prop_Addr']].drop_duplicates()
subset

In [ ]:
locator = Nominatim(user_agent="MH-ML")
geocode = RateLimiter(locator.geocode, min_delay_seconds=1.5)
locations = [geocode(addr+', Ames, Iowa') if type(addr)==str else None for addr in subset.Prop_Addr.values]

In [ ]:
lats = []
for i in range(len(locations)):
    if locations[i]==None:
        lats.append(locations[i])
        continue
    lats.append(locations[i].latitude)
subset['lat'] = lats

In [ ]:
longs = []
for i in range(len(locations)):
    if locations[i]==None:
        longs.append(locations[i])
        continue
    longs.append(locations[i].longitude)
subset['long'] = longs

In [ ]:
merged = pd.merge(housing,subset,left_on='PID',right_on='GeoRefNo',how='left')
merged.to_csv('Ames_HousePrice_withAddress.csv')